In [ ]:
# Count the number of contrast maps for each subject

import os
from collections import defaultdict

# Base path to the z-maps
base_path = "/ptmp/hmueller2/Downloads/contrast_maps_fsLR"
print("Base path:", base_path)

# Initialize a dictionary to store the count of contrast maps per subject
contrast_map_counts = defaultdict(int)

# Loop through all subjects and sessions
for subject in os.listdir(base_path):
    if subject.startswith("sub-07") and subject[4:].isdigit():  # Check subject naming pattern
        subject_path = os.path.join(base_path, subject)
        print("Subject path:", subject_path)
        if os.path.isdir(subject_path):  # Ensure it's a directory
            for session in os.listdir(subject_path):
                if session.startswith("ses-") and session[4:].isdigit():  # Check session naming pattern
                    session_path = os.path.join(subject_path, session)
                    # print("  Session path:", session_path)
                    if os.path.isdir(session_path):  # Ensure it's a directory
                        # Try to find all directories matching the pattern inside the session_path
                        for res_dir in os.listdir(session_path):
                            if res_dir.startswith("res_task-") and "fsLR" in res_dir and "dir-" in res_dir:
                                res_dir_path = os.path.join(session_path, res_dir)
                                print("    Res dir path:", res_dir_path)
                                z_score_maps_path = os.path.join(res_dir_path, "z_score_maps")
                                #print("      z_score_maps_path:", z_score_maps_path)
                                if os.path.exists(z_score_maps_path):
                                    # Count the number of .dscalar.nii files in the directory
                                    contrast_maps = [f for f in os.listdir(z_score_maps_path) if f.endswith(".dscalar.nii")]
                                    contrast_map_counts[subject] += len(contrast_maps)

# Print the results
for subject, count in contrast_map_counts.items():
    print(f"Subject {subject}: {count} contrast maps")

In [ ]:
# Count the number of contrast maps that overlap between all subjects
# Initialize a dictionary to store overlap counts
overlap_counts = defaultdict(int)

# Logic to calculate overlap between subjects will go here